In [13]:
import pandas as pd
import numpy as np

In [14]:
df = pd.read_csv('/content/diabetes.csv')

In [15]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [16]:
X = df.iloc[ : , : -1]
y = df.iloc[ : , -1] 

In [17]:
from sklearn.preprocessing import StandardScaler

In [18]:
scaler = StandardScaler()

In [19]:
X = scaler.fit_transform(X)

In [23]:
from sklearn.model_selection import train_test_split

X_train,X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

In [24]:
import tensorflow
from tensorflow import keras
from keras import Sequential
from keras.layers import Dense

In [25]:
model = Sequential()

In [26]:
model.add(Dense(32, activation = 'relu', input_dim = 8))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [27]:
model.fit(X_train, y_train, batch_size = 32, epochs = 10)

Epoch 1/10
20/20 [==============================] - 1s 3ms/step - loss: 0.6822 - accuracy: 0.5798
Epoch 2/10
20/20 [==============================] - 0s 2ms/step - loss: 0.6246 - accuracy: 0.7215
Epoch 3/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5867 - accuracy: 0.7541
Epoch 4/10
20/20 [==============================] - 0s 2ms/step - loss: 0.5592 - accuracy: 0.7622
Epoch 5/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5390 - accuracy: 0.7590
Epoch 6/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5243 - accuracy: 0.7590
Epoch 7/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5133 - accuracy: 0.7557
Epoch 8/10
20/20 [==============================] - 0s 3ms/step - loss: 0.5049 - accuracy: 0.7524
Epoch 9/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4981 - accuracy: 0.7557
Epoch 10/10
20/20 [==============================] - 0s 3ms/step - loss: 0.4915 - accuracy: 0.7557



    1. How to select the best optimizer for our model
    2. Number of nodes in a layer
    3. Number of hidden layers in model
    4. How to select all of the above parameters in one go

In [34]:
import keras_tuner as kt

<ipython-input-34-5fd8096cdee5>:1: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  import kerastuner as kt


## **Finding the best optimizer**

In [37]:
def build_model(hp):

  model = Sequential()
  model.add(Dense(32, activation = 'relu', input_dim = 8))
  model.add(Dense(1, activation = 'sigmoid'))

  optimizer = hp.Choice('optimizer' , values = ['adam', 'sgd', 'rmsprop', 'adadelta'])

  model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

In [38]:
tuner = kt.RandomSearch(build_model, 
                        objective = 'val_accuracy',
                        max_trials = 5
                        )

In [39]:
tuner.search(X_train, y_train, epochs=5, validation_data = (X_test, y_test))

Trial 4 Complete [00h 00m 01s]
val_accuracy: 0.7207792401313782

Best val_accuracy So Far: 0.7792207598686218
Total elapsed time: 00h 00m 07s


In [41]:
tuner.get_best_hyperparameters()[0].values

{'optimizer': 'adam'}

In [42]:
model = tuner.get_best_models(num_models = 1)[0]

In [44]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 32)                288       
                                                                 
 dense_1 (Dense)             (None, 1)                 33        
                                                                 
Total params: 321
Trainable params: 321
Non-trainable params: 0
_________________________________________________________________


In [46]:
model.fit(X_train, y_train, batch_size = 32, validation_data=(X_test, y_test), epochs = 50, initial_epoch=5)

Epoch 6/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4429 - accuracy: 0.7866 - val_loss: 0.4606 - val_accuracy: 0.7922
Epoch 7/50
20/20 [==============================] - 0s 5ms/step - loss: 0.4425 - accuracy: 0.7834 - val_loss: 0.4635 - val_accuracy: 0.7987
Epoch 8/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4420 - accuracy: 0.7801 - val_loss: 0.4611 - val_accuracy: 0.8052
Epoch 9/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4412 - accuracy: 0.7834 - val_loss: 0.4606 - val_accuracy: 0.7922
Epoch 10/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4405 - accuracy: 0.7883 - val_loss: 0.4603 - val_accuracy: 0.7922
Epoch 11/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4400 - accuracy: 0.7883 - val_loss: 0.4598 - val_accuracy: 0.7922
Epoch 12/50
20/20 [==============================] - 0s 4ms/step - loss: 0.4398 - accuracy: 0.7850 - val_loss: 0.4602 - val_accuracy: 0.7987
Epoch 13/50
20/20

# **Finding the number of neurons in hidden layer**

In [57]:
def build_model(hp):
  model = Sequential()

  units = hp.Int('units', min_value = 8,max_value = 128, step = 8)

  model.add(Dense(units = units, activation = 'relu', input_dim=8)
  model.add(Dense(1, activation = 'sigmoid'))

  model.compile(optimizer= 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

  return model

SyntaxError: ignored